# Database 1995 - 2019

Work with the database of all the data, possibility to extract more economy data

In [1]:
import pandas as pd
import geopandas as gpd
import json
import os
import googlemaps
from collections import defaultdict, OrderedDict
from tqdm import tqdm
import cbsodata

In [2]:
os.chdir('D:/bachelors/datasets/')

In [9]:
meta = pd.DataFrame(cbsodata.get_meta('70072ned', 'DataProperties'))
meta.to_excel('territory_bases_v3/meta.xlsx', index = None)

In [5]:
data = pd.read_csv('territory_bases_v3/data.csv', sep = ';')